<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/BP_CNN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import activations

# Load Data From Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls
!pwd

drive  sample_data
/content


# Get Data Set

In [ ]:
def getDataset(): 
  directory = '/content/drive/My Drive/Bp_dataset_v4/MelSpectrogram_128/MelSpectrogram_Mel128_AugNormal'


  train_set=tf.keras.preprocessing.image_dataset_from_directory(
      directory,
      labels="inferred",
      label_mode="categorical",
      subset="training",
      class_names=None,
      shuffle=True,
      seed = 10, #10 50 100 verecegiz 
      batch_size=32,
      image_size=(128, 128),
      validation_split = 0.2,
      #interpolation="lanczos5"
  )

  test_set = tf.keras.preprocessing.image_dataset_from_directory(
      directory,
      labels="inferred",
      label_mode="categorical",
      subset="validation",
      validation_split = 0.2,
      class_names=None,
      shuffle=True,
      seed = 10, #10 50 100 verecegiz
      batch_size=32,
      image_size=(128, 128),
      #interpolation="lanczos5"
  )
  #print(test_set)
  #for data, labels in test_set:
    #print(data.shape)  # (64, 200, 200, 3)
  return train_set, test_set

# Training Custom CNN

In [ ]:
def trainCustomCNN(train_set, test_set):
  X_train  = train_set
  opt  = tf.keras.optimizers.Adam(learning_rate=0.00001)

  #input_shape=(128, 128),
  model = keras.Sequential() 
  
  model.add(keras.layers.Conv2D(128, input_shape=(128,128,3), kernel_size=80, strides=4, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))


  model.add(keras.layers.Lambda(lambda x : keras.backend.mean(x, axis=1)))
  model.add(keras.layers.Lambda(lambda x : keras.backend.mean(x, axis=1)))
  model.add(keras.layers.Dense(8, activation='softmax'))

  model.compile(optimizer = opt, loss='categorical_crossentropy',metrics = ['accuracy', keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()]) 
  model.summary()  
  model.fit(x = X_train, validation_data = test_set, epochs = 50)

# Training CNN With Transfer Learning

In [ ]:
def transferLearning():  
  #Set pretrained Layers
  #for layer in model.layers[:150]:
  #    layer.trainable=False
  #for layer in model.layers[150:]:
  #    layer.trainable=True

  #opt = tf.keras.optimizers.SGD(learning_rate=0.001)


  opt  = tf.keras.optimizers.Adam(learning_rate=0.00001)
  for j in range(3,4):
    foldNo = 'fold' + str(j + 1)
    test_set  = testPreprocess(foldNo)
    train_set = trainPreprocess(foldNo)
    print("")
    print(foldNo)
    
    epochNo = 100     
    base_model = tf.keras.applications.ResNet101(
    include_top=False, weights='imagenet', classes=3)
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    preds=Dense(3,activation='softmax')(x) #final layer with softmax activation
    model=Model(inputs=base_model.input,outputs=preds)


    print("Epoch: " + str(epochNo) + "\n\n")
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()])
    model.fit(x = train_set, validation_data = test_set, epochs = epochNo)
    print("Egitim basariyla tamamlandi.")

# Main

In [ ]:
train_set, test_set = getDataset()
trainCustomCNN(train_set, test_set)

Found 2880 files belonging to 8 classes.
Using 2304 files for training.
Found 2880 files belonging to 8 classes.
Using 576 files for validation.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       2457728   
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 128)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 1

# Making Single Prediction

In [ ]:
"""
img_path = '/content/drive/My Drive/Hushem set1/fold1/train/02_Tapered/image_053.BMP'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
"""
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

"\nimg_path = '/content/drive/My Drive/Hushem set1/fold1/train/02_Tapered/image_053.BMP'\nimg = image.load_img(img_path, target_size=(224, 224))\nx = image.img_to_array(img)\nx = np.expand_dims(x, axis=0)\nx = preprocess_input(x)\npreds = model.predict(x)\n"

In [ ]:
"""
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/Set1_5xAugmented/fold3/test/04_Amorphous/image_032.BMP', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.argmax(result))
print(train_set.class_names)
"""

"\nimport numpy as np\nfrom keras.preprocessing import image\ntest_image = image.load_img('/content/drive/My Drive/Set1_5xAugmented/fold3/test/04_Amorphous/image_032.BMP', target_size = (64,64))\ntest_image = image.img_to_array(test_image)\ntest_image = np.expand_dims(test_image, axis = 0)\nresult = model.predict(test_image)\nprint(np.argmax(result))\nprint(train_set.class_names)\n"